In [1]:
pip install -q -U google-generativeai

In [2]:
import pathlib
import textwrap

import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata

from IPython.display import display
from IPython.display import Markdown

In [3]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [4]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/learnlm-1.5-pro-experimental
models/gemini-exp-1114
models/gemini-exp-1121


In [5]:
model = genai.GenerativeModel('gemini-pro')

In [6]:
prompt = input("Input: ")
response = model.generate_content(prompt)
Markdown(response.text)

Input: Hello


Hi there! How can I assist you today?

# Additional Functions for CFG Validation

In [13]:

import json

def save_response_to_json(response_text, file_name):
    try:
        with open(file_name, 'w') as json_file:
            json.dump({"CFG_rules": response_text}, json_file, indent=4)
        print(f"Saved response to {file_name}")
    except Exception as e:
        print(f"Error saving response to JSON: {e}")


In [12]:
def load_hardcoded_cfg(option):
    """
    Load hardcoded CFG rules from the 'Production Options' file.
    """
    hardcoded_cfg = {
        "Arithmetic Expressions": {
            "S": ["T", "T + S", "T - S"],
            "T": ["F", "F * T", "F / T"],
            "F": ["( S )", "number"]
        },
        "Control Structures": {
            "stmt": ["if expr then stmt else stmt", "while expr do stmt", "assign"],
            "assign": ["identifier = expr"],
            "expr": ["expr && term", "expr || term", "term"],
            "term": ["!factor", "factor"],
            "factor": ["true", "false", "( expr )", "identifier"]
        },
        "Function Definitions and Calls": {
            "function": ["func identifier ( params ) { stmts }"],
            "params": ["identifier", "identifier , params", "ε"],
            "stmts": ["stmt ; stmts", "ε"],
            "call": ["identifier ( args )"],
            "args": ["expr", "expr , args", "ε"]
        },
        "Assignment Operations": {
            "assign": ["identifier = expr"],
            "expr": ["identifier + identifier", "identifier - identifier", "number"],
            "number": ["digit number", "digit"],
            "digit": ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
        },
        "Boolean Expressions": {
            "bool_expr": ["bool_expr && bool_term", "bool_term"],
            "bool_term": ["bool_term || bool_factor", "bool_factor"],
            "bool_factor": ["!bool_atom", "bool_atom"],
            "bool_atom": ["true", "false", "( bool_expr )", "identifier"]
        },
        "Loop Structures": {
            "loop": ["for ( assign ; expr ; assign ) stmt"],
            "stmt": ["assign ; stmt", "if expr then stmt else stmt", "while expr do stmt"],
            "expr": ["identifier < identifier", "identifier > identifier"]
        },
        "Array Operations": {
            "array_decl": ["type identifier [ number ]"],
            "array_assign": ["identifier [ expr ] = expr"],
            "expr": ["expr + term", "term"],
            "term": ["factor * term", "factor"],
            "factor": ["number", "identifier"]
        },
        "String Operations": {
            "str_expr": ["str_expr + str_term", "str_term"],
            "str_term": ['"string_literal"', "identifier"],
            "assign": ["identifier = str_expr"]
        },
        "Basic Mathematical Functions": {
            "expr": ["sin ( term )", "cos ( term )", "log ( term )", "term"],
            "term": ["term * factor", "term / factor", "factor"],
            "factor": ["number", "identifier", "( expr )"]
        },
        "Conditional Expressions with Ternary Operator": {
            "ternary_expr": ["expr ? expr : expr"],
            "expr": ["identifier < identifier", "identifier > identifier", "expr + term", "term"],
            "term": ["factor * term", "factor"],
            "factor": ["number", "identifier"]
        }
    }
    return hardcoded_cfg.get(option, {})

In [9]:

def evaluate_generated_vs_hardcoded(generated_cfg, hardcoded_cfg):
    """
    Evaluate AI-generated CFG rules against hardcoded CFG rules.
    """
    discrepancies = []
    for key, rules in hardcoded_cfg.items():
        if key not in generated_cfg or set(generated_cfg[key]) != set(rules):
            discrepancies.append({
                "symbol": key,
                "generated": generated_cfg.get(key, []),
                "expected": rules
            })
    if not discrepancies:
        print("All rules match the hardcoded CFG!")
    else:
        print("Discrepancies found:")
        for discrepancy in discrepancies:
            print(f"Symbol: {discrepancy['symbol']}")
            print(f"Generated: {discrepancy['generated']}")
            print(f"Expected: {discrepancy['expected']}")


In [10]:

def test_generated_cfg(cfg_rules, test_cases):
    """
    Test AI-generated CFG rules against code snippets.
    """
    for idx, test_case in enumerate(test_cases, start=1):
        print(f"Testing Case {idx}: {test_case}")
        # Add CFG validation logic for the test_case
        # Example: Check if test_case follows the CFG rules
        is_valid = True  # Placeholder for validation logic
        print(f"Result: {'Valid' if is_valid else 'Invalid'}")
